In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    langchain-google-community \
    google-cloud-discoveryengine \
    google-api-python-client

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.142.0 which is incompatible.


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [2]:
PROJECT_ID = "gurkomal-playground"  
LOCATION = "us-central1"
STAGING_BUCKET = "gs://avoxi_workshop_bucket"  

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [3]:
from googleapiclient import discovery
from IPython.display import display, Markdown
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain_core import prompts
from langchain.memory import ChatMessageHistory
from vertexai.preview import reasoning_engines

In [4]:
model = "gemini-1.5-pro-001"

In [5]:
DATA_STORE_ID = "data-store_1724701968933"
LOCATION_ID = "global"

In [6]:
def search_kaggle_movies(query: str) -> str:
    """Search across records in the Kaggle Movies data set."""
    from langchain_google_community import VertexAISearchRetriever

    retriever = VertexAISearchRetriever(
        project_id=PROJECT_ID,
        data_store_id=DATA_STORE_ID,
        location_id=LOCATION_ID,
        engine_data_type=1,
        max_documents=10,
    )

    result = str(retriever.invoke(query))
    return result

In [24]:
search_kaggle_movies("Harry Potter")

'[Document(metadata={\'id\': \'dd68385e2b3e0a0ddd497d5eb1a9dcc9\', \'name\': \'projects/506365831141/locations/global/collections/default_collection/dataStores/data-store_1724701968933/branches/0/documents/dd68385e2b3e0a0ddd497d5eb1a9dcc9\'}, page_content=\'{"id": "675", "tagline": "Evil Must Be Confronted.", "runtime": "138", "budget": "150000000", "poster_path": "https://image.tmdb.org/t/p/original/5aOyriWkPec0zUDxmHFP9qMmBaj.jpg", "original_language": "en", "status": "Released", "popularity": 21.3643, "homepage": "http://www.harrypotterorderofthephoenix.com/", "belongs_to_collection": "{\\\'id\\\': 1241, \\\'name\\\': \\\'Harry Potter Collection\\\', \\\'poster_path\\\': \\\'/8Xmkc1HvCOpMlbFvVabrtr6HAsp.jpg\\\', \\\'backdrop_path\\\': \\\'/wfnMt6LGqYHcNyOfsuusw5lX3bL.jpg\\\'}", "original_title": "Harry Potter and the Order of the Phoenix", "vote_count": "5633", "spoken_languages": "[{\\\'iso_639_1\\\': \\\'en\\\', \\\'name\\\': \\\'English\\\'}]", "imdb_id": "tt0373889", "release_da

In [10]:
prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
} | prompts.ChatPromptTemplate.from_messages(
    [
        prompts.MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
        prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Initialize session history
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

Reasoning Engine with LangChain Agent

In [12]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[search_kaggle_movies],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

Test Locally

In [13]:
response = agent.query(
    input="List some sci-fi movies from the 1990s",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

Here are some sci-fi movies from the 1990s:

* **12:01** (1993): A man is caught in a time loop, repeating the same day over and over again. 
* **Alien: Resurrection** (1997): Two hundred years after her death, Ellen Ripley is cloned and must face off against a new breed of xenomorphs. 


Deploy Agent to Vertex AI

In [14]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "langchain-google-community",
        "google-cloud-discoveryengine",
    ],
)

Using bucket avoxi_workshop_bucket
Writing to gs://avoxi_workshop_bucket/reasoning_engine/reasoning_engine.pkl
Writing to gs://avoxi_workshop_bucket/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://avoxi_workshop_bucket/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/506365831141/locations/us-central1/reasoningEngines/2691991492888625152/operations/8446173618466979840
ReasoningEngine created. Resource name: projects/506365831141/locations/us-central1/reasoningEngines/2691991492888625152
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/506365831141/locations/us-central1/reasoningEngines/2691991492888625152')


#### Grant Discovery Engine Editor access to Reasoning Engine service account

* To ensure your remote agent can access relevant data, grant the Discovery Engine Editor role to the Reasoning Engine service account before sending queries. 
* The remote agent will then be able to retrieve documents from the data store that are created in Vertex AI Search.

In [15]:
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=PROJECT_ID)
response = request.execute()
project_number = response["projectNumber"]
project_number

'506365831141'

In [17]:
!echo serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com

serviceAccount:service-506365831141@gcp-sa-aiplatform-re.iam.gserviceaccount.com


Grant Editor Role

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role=roles/discoveryengine.editor

#### Test remotely deployed agent

In [19]:
response = remote_agent.query(
    input="List some sci-fi movies from the 1990s",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

Here are some sci-fi movies from the 1990s:

* **Moon 44** (1990): In a future where Earth's resources are depleted, prisoners are sent to defend a mining station on one of the last remaining mineral moons.
* **Metamorphosis: The Alien Factor** (1990): A virus from outer space transforms a bio-researcher into a bloodthirsty monster.
* **Frankenstein Unbound** (1990): A scientist from the year 2031 is transported back to 1817, where he encounters Dr. Victor Frankenstein and his creation.
* **12:01 PM** (1990): A man finds himself repeating the same hour of his life over and over again.
* **Circuitry Man** (1990): In a future where most humans live underground to escape pollution, a bodyguard and an android smuggle narcotic brain chips in a post-apocalyptic Los Angeles.
* **Class of 1999** (1990): In a dystopian future, a high school overrun by gangs brings in android teachers to restore order, but the robots have their own ideas about discipline. 
* **Spaced Invaders** (1990): A group of Martians misinterpret a radio broadcast of "War of the Worlds" as a real invasion and land in a small town on Halloween. 



In [20]:
response = remote_agent.query(
    input="Who are the actors in The Matrix?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

The main actors in The Matrix are:

* Keanu Reeves as Neo
* Laurence Fishburne as Morpheus
* Carrie-Anne Moss as Trinity 
* Hugo Weaving as Agent Smith 


In [21]:
response = remote_agent.query(
    input="Are those actors in any other movies?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

Yes, all of those actors have been in many other movies! Here are some examples:

* **Keanu Reeves:** Bill & Ted's Excellent Adventure, Speed, Point Break, John Wick,  Much Ado About Nothing 
* **Laurence Fishburne:**  Boyz n the Hood, What's Love Got to Do with It, Apocalypse Now, Contagion
* **Carrie-Anne Moss:** Memento, Chocolat, Jessica Jones, Red Planet
* **Hugo Weaving:** The Lord of the Rings (trilogy), V for Vendetta, The Hobbit (trilogy),  Priscilla, Queen of the Desert 


In [22]:
response

{'input': 'Are those actors in any other movies?',
 'intermediate_steps': [],
 'history': [{'kwargs': {'content': 'List some sci-fi movies from the 1990s',
    'type': 'human'},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'lc': 1.0},
  {'kwargs': {'content': 'Here are some sci-fi movies from the 1990s:\n\n* **12:01** (1993): A man is caught in a time loop, repeating the same day over and over again. \n* **Alien: Resurrection** (1997): Two hundred years after her death, Ellen Ripley is cloned and must face off against a new breed of xenomorphs. \n',
    'tool_calls': [],
    'type': 'ai',
    'invalid_tool_calls': []},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'lc': 1.0},
  {'kwargs': {'content': 'List some sci-fi movies from the 1990s',
    'type': 'human'},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'lc': 1.0},
  {'kwargs': {'content': 'Here are 